In [1]:
import logging
log_level = 25
logging.basicConfig(level=log_level) # root logger

import re
import pandas as pd

import sys
sys.path.append('E:/Code/chat/gdpr')


In [96]:
import fitz

import os
path = 'E:/Code/chat/gdpr/pdf/guidelines'
file_name = 'wp243_rev_01_en_D8C7A64C-9064-B4B5-543CB5093ACA4937_44100.pdf'
paper_name = "Guidelines on Data Protection Officers ('DPOs')"
full_path = os.path.join(path, file_name)
doc = fitz.open(full_path)

In [97]:
import importlib
import src.extract_from_pdf
importlib.reload(src.extract_from_pdf)
from src.extract_from_pdf import output_doc_as_text

lines_to_delete = [] 
characters_to_replace = []
characters_to_replace.append(['“', '"'])
characters_to_replace.append(['”', '"'])
characters_to_replace.append(['‘', "'"])
characters_to_replace.append(['’', "'"])
characters_to_replace.append(['', "-"])
characters_to_replace.append(['–', "-"])




document_text = output_doc_as_text(doc, start_page = 3, end_page = 0, header_size=70, footer_size=90, lines_to_delete = lines_to_delete, characters_to_replace = characters_to_replace)

file = "../tmp/dpo.md"
with open(file, 'w', encoding='utf-8') as f:
    f.write(document_text)

## Once you have the markdown file

In [2]:
import re
import pandas as pd
file_path = "../../original/dpo.md"
with open(file_path, 'r', encoding = "utf-8") as file:
    text = file.read()

lines = text.split('\n')
# get rid of empty lines
lines = [line for line in lines if line]

doc_as_array = []
notes_as_array = []
#footnote_pattern = re.compile(r'^(\[\^\d{1,2}\]:)(.*)$')
footnote_pattern = re.compile(r'^\[\^(\d{1,2})\]:(.*)$')
for entry in lines:
    footnote_match = footnote_pattern.match(entry)
    if footnote_match:
        notes_as_array.append([footnote_match.group(1), footnote_match.group(2).strip()])
    else:
        doc_as_array.append(entry)

columns = ["note_number", "text"]
df_notes = pd.DataFrame(notes_as_array, columns = columns)

In [3]:
#columns = ["section", "subsection", "point", "subpoint", "heading", "text", "section_reference"]
columns = ["section", "subsection", "point", "heading", "text", "section_reference"]
section = ""
subsection = ""
point = ""
heading = False
text = ""
section_reference = ""
section_pattern = re.compile(r'(\d+)\s+(.*)')
subsection_pattern = re.compile(r'(\d\.\d+)\.\s+(.*)') 
point_pattern = re.compile(r'(\d\.\d\.\d+)\s+(.*)') 
# subpoint_pattern = re.compile(r'^(\d+)\.\s+(.+)$') 
annex_pattern = re.compile(r'(?i)^5\s+ANNEX -\s+(.*)') # ignore capitalization

table_pattern = re.compile(r'\|')
# table_entry_number_pattern = re.compile(r'(\d+(\.\d+)?(\.\d+)?)\s+(.+)')
# table_section = re.compile(r'<td colspan=6>(\d+) - (.+)$')
# table_subsection = re.compile(r'<td colspan=6>\s*(.+)\s*')


data = []
table_data = []
annex_started = False
for entry in doc_as_array:
    section_match = section_pattern.match(entry)
    subsection_match = subsection_pattern.match(entry)
    point_match = point_pattern.match(entry)
    #subpoint_match = subpoint_pattern.match(entry)
    annex_match = annex_pattern.match(entry)
    table_match = table_pattern.match(entry)
    if annex_match:
        match = annex_match
        section = "5"
        subsection = ""
        point = ""
        #subpoint = ""
        heading = True
        text = match.group(1)
        section_reference = section
        #data.append([section, subsection, point, subpoint, heading, text, section_reference])
        data.append([section, subsection, point, heading, text, section_reference])
        annex_started = True
    elif section_match:
        match = section_match
        section = match.group(1)
        subsection = ""
        point = ""
        #subpoint = ""
        heading = True
        text = match.group(2)
        if annex_started:
            section_reference = "5." + section
        else:    
            section_reference = section
        #data.append([section, subsection, point, subpoint, heading, text, section_reference])
        data.append([section, subsection, point, heading, text, section_reference])
    elif subsection_match:
        match = subsection_match
        section = section
        subsection = match.group(1)
        point = ""
        #subpoint = ""
        heading = True
        text = match.group(2)
        section_reference = subsection
        #data.append([section, subsection, point, subpoint, heading, text, section_reference])
        data.append([section, subsection, point, heading, text, section_reference])
    elif point_match:
        match = point_match
        section = section
        subsection = subsection
        point = match.group(1)
        #subpoint = ""
        heading = True
        text = match.group(2)
        section_reference = point
        #data.append([section, subsection, point, subpoint, heading, text, section_reference])
        data.append([section, subsection, point, heading, text, section_reference])
    # elif subpoint_match:
    #     match = subpoint_match
    #     section = section
    #     subsection = subsection
    #     point = point
    #     subpoint = match.group(1)
    #     heading = False
    #     text = match.group(2)
    #     section_reference = section + "." + subsection + "." + point + ")" + subpoint
    #     data.append([section, subsection, point, subpoint, heading, text, section_reference])
    
    
    # elif table_match:
    #     pass
        # components = [component.strip() for component in entry.strip().strip('|').split('|')]
        # if len(components) == 5:
        #     table_entry_number_match = table_entry_number_pattern.match(components[0])
        #     if table_entry_number_match:
        #         components = [table_entry_number_match.group(1), table_entry_number_match.group(4)] + components[1:]
        #         table_data.append(components)
        #     else:
        #         print(f"Row removed from table_data: {components}")
        #         #components = [""] + components
        # elif len(components) == 1:
        #     table_section_match = table_section.match(components[0])
        #     table_subsection_match = table_subsection.match(components[0])
        #     if table_section_match:
        #         components = [table_section_match.group(1), table_section_match.group(2)]
        #         table_data.append(components)
        #     elif table_subsection_match:
        #         print(f"Row removed from table_data: {components}")
        #         # components = ["", table_subsection_match.group(1)]
        #     else:
        #         raise AttributeError(f"A row with one element did not match the required patterns: {entry}")
        # else:
        #     raise AttributeError("A table entry did not have the correct number of elements")
    else:
        section = section
        subsection = subsection
        point = point
        heading = False
        text = entry
        section_reference = section_reference
        
        #data.append(["", "", "", "",  heading, text, section_reference])
        data.append(["", "", "", heading, text, section_reference])




df = pd.DataFrame(data, columns = columns)
# Remove my note about the table
#df = df[df["text"] != 'Note this table contains a column "References to BCR-C, application form BCR-C, and / or supporting documents[^14]" which is empty in the document because it is supposed to be filled out by the controller'] 

In [4]:
df[40:60]
#df[df["text"] == " "]

,section,subsection,point,heading,text,section_reference
40,,,,False,- processing of real time geo-location data of...,2.1.3
41,,,,False,- processing of customer data in the regular c...,2.1.3
42,,,,False,- processing of personal data for behavioural ...,2.1.3
43,,,,False,Examples that do not constitute large-scale pr...,2.1.3
44,,,,False,- processing of patient data by an individual ...,2.1.3
45,,,,False,- processing of personal data relating to crim...,2.1.3
46,2,2.1,2.1.4,True,'REGULAR AND SYSTEMATIC MONITORING',2.1.4
47,,,,False,The notion of regular and systematic monitorin...,2.1.4
48,,,,False,"However, the notion of monitoring is not restr...",2.1.4
49,,,,False,WP29 interprets 'regular' as meaning one or mo...,2.1.4


In [5]:
# Add footnotes
import re

def find_footnote_references(text):
    pattern = r'\[\^(\d+)\]'
    return re.findall(pattern, text)

for index, row in df.iterrows():
    footnotes = find_footnote_references(row['text'])
    if footnotes:
        augmented_note = row['text']
        for note in footnotes:
            augmented_note += f"\n[^{note}]: {df_notes[df_notes['note_number'] == note].iloc[0]['text']}"
        print(f"Row {index} augmented with footnotes")
        #print(augmented_note)
        df.at[index, "text"] = augmented_note
        #print

Row 1 augmented with footnotes
Row 2 augmented with footnotes
Row 4 augmented with footnotes
Row 5 augmented with footnotes
Row 11 augmented with footnotes
Row 12 augmented with footnotes
Row 14 augmented with footnotes
Row 16 augmented with footnotes
Row 18 augmented with footnotes
Row 21 augmented with footnotes
Row 22 augmented with footnotes
Row 31 augmented with footnotes
Row 47 augmented with footnotes
Row 48 augmented with footnotes
Row 62 augmented with footnotes
Row 68 augmented with footnotes
Row 69 augmented with footnotes
Row 70 augmented with footnotes
Row 72 augmented with footnotes
Row 82 augmented with footnotes
Row 93 augmented with footnotes
Row 99 augmented with footnotes
Row 120 augmented with footnotes
Row 148 augmented with footnotes
Row 154 augmented with footnotes
Row 165 augmented with footnotes


In [6]:
file = "../../inputs/documents/dpo.parquet" # use parquet to deal with the complex text so I don't need to worry about escape characters
df.to_parquet(file, engine = 'pyarrow')

#df_no_table.to_csv(file, encoding = "utf-8", sep="|", index = False, na_rep="", quotechar='"')


## Check that the document class works as expected

In [10]:
import sys
sys.path.append('E:/Code/chat/gdpr')

import importlib
import gdpr_rag.documents.dpo
importlib.reload(gdpr_rag.documents.dpo)
from gdpr_rag.documents.dpo import DPO

path_to_manual_as_csv_file = "../../inputs/documents/dpo.parquet"

doc = DPO(path_to_manual_as_csv_file)


In [12]:
from IPython.display import Markdown, display

section = "1"

section = "2.1"
section = "2.1.3"

#section = "5.9"

print(doc.get_heading(section))
display(Markdown(doc.get_text(section)))


2 Designation of a DPO
2.1 Mandatory designation
2.1.3 'LARGE SCALE'


# 2 Designation of a DPO

## 2.1 Mandatory designation

### 2.1.3 'LARGE SCALE'

Article 37(1)(b) and (c) requires that the processing of personal data be carried out on a large scale in order for the designation of a DPO to be triggered. The GDPR does not define what constitutes large-scale processing, though recital 91 provides some guidance.[^14]

Indeed, it is not possible to give a precise number either with regard to the amount of data processed or the number of individuals concerned, which would be applicable in all situations. This does not exclude the possibility, however, that over time, a standard practice may develop for identifying in more specific and/or quantitative terms what constitutes 'large scale' in respect of certain types of common processing activities. The WP29 also plans to contribute to this development, by way of sharing and publicising examples of the relevant thresholds for the designation of a DPO.

In any event, the WP29 recommends that the following factors, in particular, be considered when determining whether the processing is carried out on a large scale:

- The number of data subjects concerned - either as a specific number or as a proportion of the relevant population

- The volume of data and/or the range of different data items being processed

- The duration, or permanence, of the data processing activity

- The geographical extent of the processing activity

Examples of large-scale processing include:

- processing of patient data in the regular course of business by a hospital  - processing of travel data of individuals using a city's public transport system (e.g. tracking via travel cards)

- processing of real time geo-location data of customers of an international fast food chain for statistical purposes by a processor specialised in providing these services

- processing of customer data in the regular course of business by an insurance company or a bank

- processing of personal data for behavioural advertising by a search engine - processing of data (content, traffic, location) by telephone or internet service providers

Examples that do not constitute large-scale processing include:

- processing of patient data by an individual physician

- processing of personal data relating to criminal convictions and offences by an individual lawyer

  
[^14]: According to the recital, 'large-scale processing operations which aim to process a considerable amount of personal data at regional, national or supranational level and which could affect a large number of data subjects and which are likely to result in a high risk' would be included, in particular. On the other hand, the recital specifically provides that 'the processing of personal data should not be considered to be on a large scale if the processing concerns personal data from patients or clients by an individual physician, other health care professional or lawyer'. It is important to consider that while the recital provides examples at the extremes of the scale (processing by an individual physician versus processing of data of a whole country or across Europe); there is a large grey zone in between these extremes. In addition, it should be borne in mind that this recital refers to data protection impact assessments. This implies that some elements might be specific to that context and do not necessarily apply to the designation of DPOs in the exact same way.